<a href="https://colab.research.google.com/github/yash261/DeepLearning/blob/main/OptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create CNN Model and Optimize it using Keras Tuner

In [11]:
!pip install keras-tuner

In [13]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [14]:
print(tf.__version__)

2.4.1


In [15]:
fashion_mnist=keras.datasets.fashion_mnist
fashion_mnist.load_data()

((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

In [16]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [17]:
train_images=train_images/255.0
test_images=test_images/255.0

In [18]:
train_images[0].shape

(28, 28)

In [19]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [20]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [21]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [22]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [23]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 19s]
val_accuracy: 0.8676666617393494

Best val_accuracy So Far: 0.9070000052452087
Total elapsed time: 00h 01m 43s
INFO:tensorflow:Oracle triggered exit


In [24]:
model=tuner_search.get_best_models(num_models=1)[0]

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        76832     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               1734768   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 1,813,690
Trainable params: 1,813,690
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1406 - accuracy: 0.9477 - val_loss: 0.2633 - val_accuracy: 0.9132
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1006 - accuracy: 0.9615 - val_loss: 0.2737 - val_accuracy: 0.9118
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0686 - accuracy: 0.9755 - val_loss: 0.3275 - val_accuracy: 0.9150
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0480 - accuracy: 0.9829 - val_loss: 0.3697 - val_accuracy: 0.9103
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0348 - accuracy: 0.9874 - val_loss: 0.4221 - val_accuracy: 0.9157
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0294 - accuracy: 0.9898 - val_loss: 0.4438 - val_accuracy: 0.9113
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0203 - accuracy: 0.9929 - val_loss: 0.4992 - val_accuracy

In [27]:
test_labels

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

In [29]:
pr=model.predict(test_images)

In [32]:
for i in pr[:10]:
  print(np.argmax(i+1))

9
2
1
1
6
1
4
6
5
7
